# Model 8b - MLPClassifier with standardization & hyperparameter tuning (5-fold cross-validation)

In [1]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
import math
import os

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_validate, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, f1_score, r2_score

from openpyxl import load_workbook

In [2]:
# get data
parentdir = os.path.dirname(os.getcwd())
data = pd.read_excel(parentdir+'\\Data\\SWedge Results.xlsx',sheet_name = "Probabilistic Values", engine='openpyxl')
data.head()

,Wedge ID,Safety Factor,Ln Safety Factor,Safety Factor Class,Safety Factor Class_1,Safety Factor Class_2,Ln Safety Factor Class_2,Wedge Volume (m3),Wedge Weight (MN),Plunge Line of Intersection (°),...,Water Pressure Joint 2 (MPa),Water Pressure Basal Joint (MPa),Water Pressure Tension Crack (MPa),Ponded Water Depth (m),Seismic Alpha,Seismic Plunge (°),Seismic Trend (°),Maximum Persistence Joint 1 (m),Maximum Persistence Joint 2 (m),Maximum Persistence Basal Joint (m)
0,BFA 60 [0],1.082239,0.079032,11.0,6.0,1.05,0.038481,6349.248707,171.429715,31.846178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
1,BFA 60 [1],1.203906,0.185571,13.0,7.0,1.35,0.293893,7701.148241,207.931003,31.905513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
2,BFA 60 [2],0.896601,-0.109144,9.0,5.0,0.75,-0.308093,2341.641868,63.224330,36.973415,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
3,BFA 60 [3],0.680996,-0.384199,7.0,4.0,0.75,-0.308093,155.345062,4.194317,54.969435,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN
4,BFA 60 [4],1.263948,0.234240,13.0,7.0,1.35,0.293893,7468.340623,201.645197,29.688564,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN


In [3]:
print(np.shape(data))

(5000, 92)


In [4]:
# get data specifically for the modelling (i.e., the inputs and output)
data_model = data[["Safety Factor Class_2","Dip of Joint 1 (°)","Dip Direction of Joint 1 (°)","Dip of Joint 2 (°)","Dip Direction of Joint 2 (°)","Dip of Slope (°)","Dip Direction of Slope (°)","Friction Angle of Joint 1 (°)","Friction Angle of Joint 2 (°)"]]
print(np.shape(data_model))
data_model.head()

(5000, 9)


,Safety Factor Class_2,Dip of Joint 1 (°),Dip Direction of Joint 1 (°),Dip of Joint 2 (°),Dip Direction of Joint 2 (°),Dip of Slope (°),Dip Direction of Slope (°),Friction Angle of Joint 1 (°),Friction Angle of Joint 2 (°)
0,1.05,39.265808,120.865923,51.646228,221.979277,58.840543,182.626968,29.567773,29.522638
1,1.35,38.981309,128.836961,57.766382,235.428421,63.804918,181.820235,32.713619,29.079492
2,0.75,42.032968,117.504566,62.427355,217.726775,58.134485,180.398207,29.660213,27.455866
3,0.75,69.264568,137.906910,66.183726,246.195109,61.968796,182.439496,30.866657,34.401616
4,1.35,46.728166,121.226945,50.803809,241.060589,60.832522,179.091174,28.789453,28.613525


In [5]:
# remove any realizations that are not kinematically possible and any duplicates
data_model =  data_model.dropna()
data_model = data_model.drop_duplicates()
print(np.shape(data_model))

(4992, 9)


In [6]:
data_model['Safety Factor Class_2'] = data_model['Safety Factor Class_2'].apply(str)

### Hyperparameter tuning

In [7]:
# hyperparameter grid for MLPClassifier
# number of neurons in hidden layer(s)
# note: 1 hidden layer - > (x,), 2 hidden layers -> (x,y), 3 hidden layers -> (x,y,z), etc
hidden_layer_sizes = [(10,),(15,),(20,),(10,10),(15,10),(20,10)]

# activation function for hidden layer
activation = ['tanh','relu']

# solver for weight optimization
solver = ['sgd','adam']

# alpha (strength of L2 regularization term)
# https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mlp_alpha.html
alpha = [0.0001,0.001,0.01,0.1,1]

# initial learning rate
learning_rate_init = [0.001,0.01,0.1]

# maximum number of iterations
max_iter = [200,500]


# create the random grid
param_grid = {'mlpclassifier__hidden_layer_sizes': hidden_layer_sizes,
              'mlpclassifier__activation': activation,
              'mlpclassifier__solver': solver,
              'mlpclassifier__alpha': alpha,
              'mlpclassifier__learning_rate_init': learning_rate_init,
              'mlpclassifier__max_iter': max_iter}

print(param_grid)

{'mlpclassifier__hidden_layer_sizes': [(10,), (15,), (20,), (10, 10), (15, 10), (20, 10)], 'mlpclassifier__activation': ['tanh', 'relu'], 'mlpclassifier__solver': ['sgd', 'adam'], 'mlpclassifier__alpha': [0.0001, 0.001, 0.01, 0.1, 1], 'mlpclassifier__learning_rate_init': [0.001, 0.01, 0.1], 'mlpclassifier__max_iter': [200, 500]}


In [8]:
# random_state=123, early_stopping=True, validation_fraction=0.2

# function to get train & test R2 and RMSE for specified dataset size where hyperparameter tuning was performed
def hyperparam_results(data, dataset_size, param_grid):
    random_state_val = [0,1,42,123]
    param_grid = param_grid
    
    accuracy_train_subsample_list = []
    f1_train_subsample_list = []
    accuracy_test_subsample_list = []
    f1_test_subsample_list = []

    r2_train_subsample_list = []
    r2_test_subsample_list = []
    rmse_train_subsample_list = []
    rmse_test_subsample_list = []


    for x in range(0,4):
        # get subsample of data
        data_subsample = data_model.sample(n = dataset_size,random_state = 1)

        # train/test split with different random_state values (0, 1, 42, and 123)
        train_subsample, test_subsample = train_test_split(data_subsample, test_size=0.2, random_state=random_state_val[x])

        x_train_subsample = train_subsample[["Dip of Joint 1 (°)","Dip Direction of Joint 1 (°)","Dip of Joint 2 (°)","Dip Direction of Joint 2 (°)","Dip of Slope (°)","Dip Direction of Slope (°)","Friction Angle of Joint 1 (°)","Friction Angle of Joint 2 (°)"]]
        y_train_subsample = train_subsample[["Safety Factor Class_2"]]
        y_train_subsample = np.ravel(y_train_subsample)
        
        x_test_subsample = test_subsample[["Dip of Joint 1 (°)","Dip Direction of Joint 1 (°)","Dip of Joint 2 (°)","Dip Direction of Joint 2 (°)","Dip of Slope (°)","Dip Direction of Slope (°)","Friction Angle of Joint 1 (°)","Friction Angle of Joint 2 (°)"]]
        y_test_subsample = test_subsample[["Safety Factor Class_2"]].values
        y_test_subsample = np.ravel(y_test_subsample)

        # train mlp model with hyperparameter tuning (5-fold cross-validation w/ randomized search)
        # make pipeline for mlp with pre-processing (standardizing the data)
        pipe_mlp = make_pipeline(StandardScaler(), MLPClassifier(random_state = 123,early_stopping=True,validation_fraction=0.2))
        random_search = RandomizedSearchCV(estimator=pipe_mlp, param_distributions=param_grid,n_iter=100, n_jobs=-1,cv=5,random_state=123)

        random_search.fit(x_train_subsample, y_train_subsample)
        
        ypred_mlp = random_search.predict(x_train_subsample)
        ypred_mlp = np.reshape(ypred_mlp,(len(ypred_mlp),1))

        # training accuracy, f1, r2, and rmse
        accuracy_train_subsample = random_search.score(x_train_subsample,y_train_subsample)
        f1_train_subsample = f1_score(y_train_subsample,ypred_mlp,average='macro')

        r2_train_subsample = r2_score(y_train_subsample,ypred_mlp)
        rmse_train_subsample = math.sqrt(mean_squared_error(y_train_subsample,ypred_mlp))
        
        # append training accuracy, f1, r2, and rmse to their respective lists
        accuracy_train_subsample_list.append(accuracy_train_subsample)
        f1_train_subsample_list.append(f1_train_subsample)
       
        r2_train_subsample_list.append(r2_train_subsample)
        rmse_train_subsample_list.append(rmse_train_subsample)
    
        # test the tuned mlp model
        # predict y test
        ypred_test_mlp = random_search.predict(x_test_subsample)
        ypred_test_mlp = np.reshape(ypred_test_mlp,(len(ypred_test_mlp),1))

        # test accuracy, f1, r2, and rmse
        accuracy_test_subsample = random_search.score(x_test_subsample,y_test_subsample)
        f1_test_subsample = f1_score(y_test_subsample,ypred_test_mlp,average='macro')

        r2_test_subsample = r2_score(y_test_subsample,ypred_test_mlp)
        rmse_test_subsample = math.sqrt(mean_squared_error(y_test_subsample,ypred_test_mlp))
        
        # append test accuracy, f1, r2, and rmse to their respective lists
        accuracy_test_subsample_list.append(accuracy_test_subsample)
        f1_test_subsample_list.append(f1_test_subsample)
        
        r2_test_subsample_list.append(r2_test_subsample)
        rmse_test_subsample_list.append(rmse_test_subsample)
        
    return accuracy_train_subsample_list, f1_train_subsample_list, r2_train_subsample_list, rmse_train_subsample_list, accuracy_test_subsample_list, f1_test_subsample_list, r2_test_subsample_list, rmse_test_subsample_list
    

In [9]:
# MLP model results for dataset size = 100 data points
accuracy_train_100, f1_train_100, r2_train_100, rmse_train_100, accuracy_test_100, f1_test_100, r2_test_100, rmse_test_100 = hyperparam_results(data_model,100,param_grid)

C:\Users\bever\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages

In [10]:
# training and test accuracy for MLP trained on 100 data points for four different random_state values in train/test split
print(accuracy_train_100)
print(accuracy_test_100)

[0.8125, 0.825, 0.95, 0.9]
[0.45, 0.5, 0.4, 0.55]


In [11]:
# training and test f1 for MLP trained on 100 data points for four different random_state values in train/test split
print(f1_train_100)
print(f1_test_100)

[0.5420677502310155, 0.6410880785880786, 0.7944698510114331, 0.6969327491504911]
[0.18727106227106227, 0.21036106750392464, 0.20476190476190473, 0.3068783068783069]


In [12]:
# training and test r2 for MLP trained on 100 data points for four different random_state values in train/test split
print(r2_train_100)
print(r2_test_100)

[0.7594504811890637, 0.7264930343655466, 0.8171177232314025, 0.8792715367301003]
[0.3310532722740712, 0.05991855730075668, 0.3684210526315792, 0.7156398104265402]


In [13]:
# training and test rmse for MLP trained on 100 data points for four different random_state values in train/test split
print(rmse_train_100)
print(rmse_test_100)

[0.20432816741702547, 0.2551960031034969, 0.1987460691435179, 0.17175564037317667]
[0.5371219600798313, 0.4494441010848846, 0.4189272013130682, 0.23237900077244503]


In [41]:
# train - test accuracy for MLP trained on 100 data points for four different random_state values in train/test split
accuracy_diff_100 = np.asarray(accuracy_train_100) - np.asarray(accuracy_test_100)
accuracy_diff_100

array([0.3625, 0.325 , 0.55  , 0.35  ])

In [43]:
# train - test f1 for MLP trained on 100 data points for four different random_state values in train/test split
f1_diff_100 = np.asarray(f1_train_100) - np.asarray(f1_test_100)
f1_diff_100

array([0.35479669, 0.43072701, 0.58970795, 0.39005444])

In [44]:
# train - test r2 for MLP trained on 100 data points for four different random_state values in train/test split
r2_diff_100 = np.asarray(r2_train_100) - np.asarray(r2_test_100)
r2_diff_100

array([0.42839721, 0.66657448, 0.44869667, 0.16363173])

In [45]:
# test - train rmse for MLP trained on 100 data points for four different random_state values in train/test split
rmse_diff_100 = np.asarray(rmse_test_100) - np.asarray(rmse_train_100)
rmse_diff_100

array([0.33279379, 0.1942481 , 0.22018113, 0.06062336])

In [14]:
# MLP model results for dataset size = 150 data points
accuracy_train_150, f1_train_150, r2_train_150, rmse_train_150, accuracy_test_150, f1_test_150, r2_test_150, rmse_test_150 = hyperparam_results(data_model,150,param_grid)

C:\Users\bever\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages

In [15]:
# training and test accuracy for MLP trained on 150 data points for four different random_state values in train/test split
print(accuracy_train_150)
print(accuracy_test_150)

[0.8333333333333334, 0.9, 0.775, 0.9583333333333334]
[0.5666666666666667, 0.4666666666666667, 0.6, 0.7]


In [16]:
# training and test f1 for MLP trained on 150 data points for four different random_state values in train/test split
print(f1_train_150)
print(f1_test_150)

[0.4681241411252576, 0.5533387445887445, 0.4307005494505494, 0.84158141304993]
[0.27308006535947715, 0.38936839557957575, 0.3384615384615385, 0.4313840155945419]


In [17]:
# training and test r2 for MLP trained on 150 data points for four different random_state values in train/test split
print(r2_train_150)
print(r2_test_150)

[0.7546344253930952, 0.9011466982997233, 0.719783332683851, 0.9773049645390071]
[0.5501220008133387, 0.6962025316455696, 0.5853052809146713, 0.5770676691729322]


In [18]:
# training and test rmse for MLP trained on 150 data points for four different random_state values in train/test split
print(rmse_train_150)
print(rmse_test_150)

[0.20996031371031368, 0.15811388300841894, 0.25478749315197297, 0.07745966692414835]
[0.443471156521669, 0.21908902300206645, 0.31358146203711296, 0.21213203435596428]


In [46]:
# train - test accuracy for MLP trained on 150 data points for four different random_state values in train/test split
accuracy_diff_150 = np.asarray(accuracy_train_150) - np.asarray(accuracy_test_150)
accuracy_diff_150

array([0.26666667, 0.43333333, 0.175     , 0.25833333])

In [47]:
# train - test f1 for MLP trained on 150 data points for four different random_state values in train/test split
f1_diff_150 = np.asarray(f1_train_150) - np.asarray(f1_test_150)
f1_diff_150

array([0.19504408, 0.16397035, 0.09223901, 0.4101974 ])

In [48]:
# train - test r2 for MLP trained on 150 data points for four different random_state values in train/test split
r2_diff_150 = np.asarray(r2_train_150) - np.asarray(r2_test_150)
r2_diff_150

array([0.20451242, 0.20494417, 0.13447805, 0.4002373 ])

In [49]:
# test - train rmse for MLP trained on 150 data points for four different random_state values in train/test split
rmse_diff_150 = np.asarray(rmse_test_150) - np.asarray(rmse_train_150)
rmse_diff_150

array([0.23351084, 0.06097514, 0.05879397, 0.13467237])

In [19]:
# MLP model results for dataset size = 200 data points
accuracy_train_200, f1_train_200, r2_train_200, rmse_train_200, accuracy_test_200, f1_test_200, r2_test_200, rmse_test_200 = hyperparam_results(data_model,200,param_grid)

C:\Users\bever\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages

In [20]:
# training and test accuracy for MLP trained on 200 data points for four different random_state values in train/test split
print(accuracy_train_200)
print(accuracy_test_200)

[0.78125, 0.825, 0.79375, 0.75]
[0.55, 0.625, 0.55, 0.725]


In [21]:
# training and test f1 for MLP trained on 200 data points for four different random_state values in train/test split
print(f1_train_200)
print(f1_test_200)

[0.47465338891462094, 0.5815991838336704, 0.4662791707776228, 0.3689795015722702]
[0.4121594308351524, 0.25236870811905876, 0.24858870967741936, 0.4518518518518519]


In [22]:
# training and test r2 for MLP trained on 200 data points for four different random_state values in train/test split
print(r2_train_200)
print(r2_test_200)

[0.8364677023712184, 0.8718680156815125, 0.8412035102381947, 0.797005290056487]
[0.5845954132410212, 0.8113981139811398, 0.5980369245150736, 0.5997140814867761]


In [23]:
# training and test rmse for MLP trained on 200 data points for four different random_state values in train/test split
print(rmse_train_200)
print(rmse_test_200)

[0.203100960115899, 0.16937384685954318, 0.18874586088176873, 0.23035298999578885]
[0.23237900077244503, 0.21447610589527216, 0.31104662029991575, 0.17748239349298853]


In [53]:
# train - test accuracy for MLP trained on 200 data points for four different random_state values in train/test split
accuracy_diff_200 = np.asarray(accuracy_train_200) - np.asarray(accuracy_test_200)
print(accuracy_diff_200)

# train - test f1 for MLP trained on 200 data points for four different random_state values in train/test split
f1_diff_200 = np.asarray(f1_train_200) - np.asarray(f1_test_200)
print(f1_diff_200)

# train - test r2 for MLP trained on 200 data points for four different random_state values in train/test split
r2_diff_200 = np.asarray(r2_train_200) - np.asarray(r2_test_200)
print(r2_diff_200)

# test - train rmse for MLP trained on 200 data points for four different random_state values in train/test split
rmse_diff_200 = np.asarray(rmse_test_200) - np.asarray(rmse_train_200)
print(rmse_diff_200)

[0.23125 0.2     0.24375 0.025  ]
[ 0.06249396  0.32923048  0.21769046 -0.08287235]
[0.25187229 0.0604699  0.24316659 0.19729121]
[ 0.02927804  0.04510226  0.12230076 -0.0528706 ]


In [24]:
# MLP model results for dataset size = 250 data points
accuracy_train_250, f1_train_250, r2_train_250, rmse_train_250, accuracy_test_250, f1_test_250, r2_test_250, rmse_test_250 = hyperparam_results(data_model,250,param_grid)

C:\Users\bever\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages

In [25]:
# training and test accuracy for MLP trained on 250 data points for four different random_state values in train/test split
print(accuracy_train_250)
print(accuracy_test_250)

[0.735, 0.805, 0.725, 0.835]
[0.6, 0.64, 0.54, 0.68]


In [26]:
# training and test f1 for MLP trained on 250 data points for four different random_state values in train/test split
print(f1_train_250)
print(f1_test_250)

[0.35741307749880924, 0.5212046055796056, 0.27333253702818916, 0.5537234474233498]
[0.2275081595004967, 0.31063988095238093, 0.2883838383838384, 0.2891158826118176]


In [27]:
# training and test r2 for MLP trained on 250 data points for four different random_state values in train/test split
print(r2_train_250)
print(r2_test_250)

[0.5472445169997943, 0.7720139327476987, 0.5955365967851012, 0.873037039138638]
[0.3958463484130492, 0.7580645161290323, 0.6171967020023557, 0.6175658532945304]


In [28]:
# training and test rmse for MLP trained on 250 data points for four different random_state values in train/test split
print(rmse_train_250)
print(rmse_test_250)

[0.2994160984315973, 0.21828879952943073, 0.3054504869860252, 0.1638596960817394]
[0.3982461550347975, 0.23237900077244503, 0.21633307652783934, 0.286356421265527]


In [52]:
# train - test accuracy for MLP trained on 250 data points for four different random_state values in train/test split
accuracy_diff_250 = np.asarray(accuracy_train_250) - np.asarray(accuracy_test_250)
print(accuracy_diff_250)

# train - test f1 for MLP trained on 250 data points for four different random_state values in train/test split
f1_diff_250 = np.asarray(f1_train_250) - np.asarray(f1_test_250)
print(f1_diff_250)

# train - test r2 for MLP trained on 250 data points for four different random_state values in train/test split
r2_diff_250 = np.asarray(r2_train_250) - np.asarray(r2_test_250)
print(r2_diff_250)

# test - train rmse for MLP trained on 250 data points for four different random_state values in train/test split
rmse_diff_250 = np.asarray(rmse_test_250) - np.asarray(rmse_train_250)
print(rmse_diff_250)

[0.135 0.165 0.185 0.155]
[ 0.12990492  0.21056472 -0.0150513   0.26460756]
[ 0.15139817  0.01394942 -0.02166011  0.25547119]
[ 0.09883006  0.0140902  -0.08911741  0.12249673]


In [29]:
# MLP model results for dataset size = 750 data points
accuracy_train_750, f1_train_750, r2_train_750, rmse_train_750, accuracy_test_750, f1_test_750, r2_test_750, rmse_test_750 = hyperparam_results(data_model,750,param_grid)

C:\Users\bever\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages

In [30]:
# training and test accuracy for MLP trained on 750 data points for four different random_state values in train/test split
print(accuracy_train_750)
print(accuracy_test_750)

[0.87, 0.91, 0.9266666666666666, 0.9133333333333333]
[0.8733333333333333, 0.86, 0.8866666666666667, 0.8666666666666667]


In [31]:
# training and test f1 for MLP trained on 750 data points for four different random_state values in train/test split
print(f1_train_750)
print(f1_test_750)

[0.6153132959073898, 0.6365934449085269, 0.630473993818772, 0.6832370958887254]
[0.7280659808381582, 0.61178646707102, 0.5631666327499661, 0.6806412623803928]


In [32]:
# training and test r2 for MLP trained on 750 data points for four different random_state values in train/test split
print(r2_train_750)
print(r2_test_750)

[0.9031036116813602, 0.9269196069582256, 0.9530446375625198, 0.9379319122375837]
[0.932373467197836, 0.9201680553952033, 0.9531572222613637, 0.936633997972288]


In [33]:
# training and test rmse for MLP trained on 750 data points for four different random_state values in train/test split
print(rmse_train_750)
print(rmse_test_750)

[0.1379613472438325, 0.12055427546683416, 0.09165151389911678, 0.1089342309224546]
[0.1067707825203131, 0.11224972160321824, 0.10456258094238749, 0.10954451150103323]


In [54]:
# train - test accuracy for MLP trained on 750 data points for four different random_state values in train/test split
accuracy_diff_750 = np.asarray(accuracy_train_750) - np.asarray(accuracy_test_750)
print(accuracy_diff_750)

# train - test f1 for MLP trained on 750 data points for four different random_state values in train/test split
f1_diff_750 = np.asarray(f1_train_750) - np.asarray(f1_test_750)
print(f1_diff_750)

# train - test r2 for MLP trained on 750 data points for four different random_state values in train/test split
r2_diff_750 = np.asarray(r2_train_750) - np.asarray(r2_test_750)
print(r2_diff_750)

# test - train rmse for MLP trained on 750 data points for four different random_state values in train/test split
rmse_diff_750 = np.asarray(rmse_test_750) - np.asarray(rmse_train_750)
print(rmse_diff_750)

[-0.00333333  0.05        0.04        0.04666667]
[-0.11275268  0.02480698  0.06730736  0.00259583]
[-0.02926986  0.00675155 -0.00011258  0.00129791]
[-0.03119056 -0.00830455  0.01291107  0.00061028]


In [34]:
# MLP model results for dataset size = 2000 data points
accuracy_train_2000, f1_train_2000, r2_train_2000, rmse_train_2000, accuracy_test_2000, f1_test_2000, r2_test_2000, rmse_test_2000 = hyperparam_results(data_model,2000,param_grid)

C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
C:\Users\bever\anaconda3\lib\site-packages\sklearn\metrics\_regression.py:96: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.

In [35]:
# training and test accuracy for MLP trained on 2000 data points for four different random_state values in train/test split
print(accuracy_train_2000)
print(accuracy_test_2000)

[0.95125, 0.938125, 0.959375, 0.93125]
[0.9025, 0.9, 0.9225, 0.9125]


In [36]:
# training and test f1 for MLP trained on 2000 data points for four different random_state values in train/test split
print(f1_train_2000)
print(f1_test_2000)

[0.824761759917753, 0.6973564998181045, 0.8071732564346895, 0.6025404636017492]
[0.6776275909858499, 0.6514562925259408, 0.7156862717278015, 0.6644690414255633]


In [37]:
# training and test r2 for MLP trained on 2000 data points for four different random_state values in train/test split
print(r2_train_2000)
print(r2_test_2000)

[0.9674511674407311, 0.9632598928701103, 0.9751324102405783, 0.9289852379650062]
[0.9553652095142706, 0.9157389170344312, 0.9510836998658878, 0.9172314177982683]


In [38]:
# training and test rmse for MLP trained on 2000 data points for four different random_state values in train/test split
print(rmse_train_2000)
print(rmse_test_2000)

[0.07685213074469699, 0.0823862852664204, 0.06832825184358224, 0.11527141883398503]
[0.09656603957913983, 0.12845232578665128, 0.09486832980505137, 0.12409673645990854]


In [55]:
# train - test accuracy for MLP trained on 2000 data points for four different random_state values in train/test split
accuracy_diff_2000 = np.asarray(accuracy_train_2000) - np.asarray(accuracy_test_2000)
print(accuracy_diff_2000)

# train - test f1 for MLP trained on 2000 data points for four different random_state values in train/test split
f1_diff_2000 = np.asarray(f1_train_2000) - np.asarray(f1_test_2000)
print(f1_diff_2000)

# train - test r2 for MLP trained on 2000 data points for four different random_state values in train/test split
r2_diff_2000 = np.asarray(r2_train_2000) - np.asarray(r2_test_2000)
print(r2_diff_2000)

# test - train rmse for MLP trained on 2000 data points for four different random_state values in train/test split
rmse_diff_2000 = np.asarray(rmse_test_2000) - np.asarray(rmse_train_2000)
print(rmse_diff_2000)

[0.04875  0.038125 0.036875 0.01875 ]
[ 0.14713417  0.04590021  0.09148698 -0.06192858]
[0.01208596 0.04752098 0.02404871 0.01175382]
[0.01971391 0.04606604 0.02654008 0.00882532]
